In [1]:
import numpy as np

from mnist import MNIST
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import ExtraTreesClassifier as ETC
from sklearn.metrics import accuracy_score

N_CLASSES = 10

print("Loading dataset...")
mndata = MNIST("./data/")
images, labels = mndata.load_training()
images = np.array(images)
labels = np.array(labels)
print("Done")

Loading dataset...
Done


In [2]:
print(images.shape)
print(labels.shape)

(60000, 784)
(60000,)


In [3]:
class ForestLayer:
    def __init__(self, clfs):
        self.clfs = clfs # list of classifiers
    
    def train(self, X, y):
        [clf.fit(X, y) for clf in self.clfs]
        
    def predict_proba(self, X):
        return [clf.predict_proba(X) for clf in self.clfs]

In [4]:
class cascadeForest:
    def __init__(self, layers):
        self.layers = layers # list of classifiers
        
    def train(self, X, y):
        acc_arr = []
        X_new = np.copy(X)
        acc_old = 0
        for i, layer in enumerate(self.layers):
            layer.train(X_new,y)
            pred = layer.predict_proba(X_new)
            
            ## verbose
            ave = np.average(np.array(pred), axis = 0)
            predictions = np.argmax(ave, axis=1)
            acc = accuracy_score(predictions, y)
            acc_arr.append(acc)
            print("Layer",i+1,"accuracy:", acc)
            ##
            if acc<=acc_old:
                break
            acc_old=acc
            
            X_new = np.concatenate(pred, axis=1)
            X_new = np.concatenate((X_new, X), axis=1)
            
    def predict_proba(self, X, y):
        acc_arr = []
        X_new = np.copy(X)
        for i, layer in enumerate(self.layers):
            pred = layer.predict_proba(X_new)
            
            ## verbose
            ave = np.average(np.array(pred), axis = 0)
            predictions = np.argmax(ave, axis=1)
            acc = accuracy_score(predictions, y)
            acc_arr.append(acc)
            print("Layer",i+1,"accuracy:", acc)
            ##
            
            X_new = np.concatenate(pred, axis=1)
            X_new = np.concatenate((X_new, X), axis=1)
            
        return np.average(np.array(pred), axis = 0), acc_arr

In [5]:
n_layers = 100 #max_num_layers
layers = []
for i in range(n_layers):thor
    clfs = []
    clfs.append( RFC(n_estimators=500, max_leaf_nodes = 100, n_jobs=6) )
    clfs.append( RFC(n_estimators=500, max_leaf_nodes = 100, n_jobs=6) )
    clfs.append( ETC(n_estimators=500, max_leaf_nodes = 100, n_jobs=6) )
    clfs.append( ETC(n_estimators=500, max_leaf_nodes = 100, n_jobs=6) )
    layers.append( ForestLayer(clfs) )

cf = cascadeForest(layers)
    
# Train on the first 10000 images:
train_x = images[:48000]
train_y = labels[:48000]

train_acc = cf.train(train_x, train_y)

Layer 1 accuracy: 0.9154375
Layer 2 accuracy: 0.9324583333333333
Layer 3 accuracy: 0.939875
Layer 4 accuracy: 0.9439583333333333
Layer 5 accuracy: 0.9474791666666667
Layer 6 accuracy: 0.9497291666666666
Layer 7 accuracy: 0.9515625
Layer 8 accuracy: 0.9534583333333333
Layer 9 accuracy: 0.9556666666666667
Layer 10 accuracy: 0.957875
Layer 11 accuracy: 0.9599375
Layer 12 accuracy: 0.9610208333333333
Layer 13 accuracy: 0.9623125
Layer 14 accuracy: 0.9639583333333334
Layer 15 accuracy: 0.9657708333333334
Layer 16 accuracy: 0.9669791666666666
Layer 17 accuracy: 0.9679375
Layer 18 accuracy: 0.9687708333333334
Layer 19 accuracy: 0.96925
Layer 20 accuracy: 0.9698125
Layer 21 accuracy: 0.9710833333333333
Layer 22 accuracy: 0.9722916666666667
Layer 23 accuracy: 0.9731875
Layer 24 accuracy: 0.9734166666666667
Layer 25 accuracy: 0.9740625
Layer 26 accuracy: 0.9747083333333333
Layer 27 accuracy: 0.975375
Layer 28 accuracy: 0.9759791666666666
Layer 29 accuracy: 0.9765833333333334
Layer 30 accuracy: 0

In [6]:
test_x = images[-12000:]
test_y = labels[-12000:]
cf = cascadeForest(layers)
pred, val_acc = cf.predict_proba(test_x,test_y)

Layer 1 accuracy: 0.9239166666666667
Layer 2 accuracy: 0.93275
Layer 3 accuracy: 0.9360833333333334
Layer 4 accuracy: 0.9388333333333333
Layer 5 accuracy: 0.9405833333333333
Layer 6 accuracy: 0.9419166666666666
Layer 7 accuracy: 0.94325
Layer 8 accuracy: 0.9435833333333333
Layer 9 accuracy: 0.945
Layer 10 accuracy: 0.94625
Layer 11 accuracy: 0.9470833333333334
Layer 12 accuracy: 0.94775
Layer 13 accuracy: 0.9479166666666666
Layer 14 accuracy: 0.9485
Layer 15 accuracy: 0.9494166666666667
Layer 16 accuracy: 0.9495
Layer 17 accuracy: 0.9494166666666667
Layer 18 accuracy: 0.9499166666666666
Layer 19 accuracy: 0.95025
Layer 20 accuracy: 0.9505833333333333
Layer 21 accuracy: 0.9508333333333333
Layer 22 accuracy: 0.9505
Layer 23 accuracy: 0.95075
Layer 24 accuracy: 0.9508333333333333
Layer 25 accuracy: 0.9510833333333333
Layer 26 accuracy: 0.9516666666666667
Layer 27 accuracy: 0.9518333333333333
Layer 28 accuracy: 0.9519166666666666
Layer 29 accuracy: 0.9518333333333333
Layer 30 accuracy: 0.9

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.